# AR6-based forcing. Part 1

Based on `12_make-forcing.py` and `13_radiative-forcing-barchart.py`
in https://github.com/ClimateIndicator/forcing-timeseries

In [ ]:
%cd ..

In [3]:
import pathlib
import numpy as np
import pandas as pd
from scipy import stats, optimize
from mce.core import ScenarioBase

In [4]:
outpath = 'datain/ds_historical.h5'
ds = ScenarioBase(outpath=outpath)

[2025-04-18 09:35:06 mce.core] INFO:file datain/ds_historical.h5 opened with mode=w


In [6]:
path_indicator = pathlib.Path('../ClimateIndicator')

In [2]:
repo = RetrieveGitHub('ClimateIndicator', 'forcing-timeseries', 'datain')

In [3]:
outpath = 'untracked/datain.h5'
dname = 'ClimateIndicator-2024/forcing-timeseries'

with h5py.File(outpath, mode='a') as h5f:
    if dname in h5f:
        grp = h5f[dname]
        grp.clear()
    else:
        grp = h5f.create_group(dname)

## Uncertainty ranges

In [7]:
SAMPLES = 100000
NINETY_TO_ONESIGMA = stats.norm.ppf(0.95)
# Required adjustment to each species to ensure overall halogenated gas ERF uncertainty is around 19%
HALOGEN_SCALING = 2.05

In [8]:
species_halogen = [
    # hydrofluorocarbons (HFC) and fully fluorinated species including perfluorocarbons (PFCs)
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23',
    'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc', 'HFC-43-10mee',
    'NF3', 'C2F6', 'C3F8', 'n-C4F10', 'n-C5F12', 'n-C6F14', 'i-C6F14',
    'C7F16', 'C8F18', 'CF4', 'c-C4F8', 'SF6', 'SO2F2', 'CCl4',
    # chlorofluorocarbons (CFC), hydrofluorochlorocarbons (HCFC),
    # chlorocarbons, hydrochlorocarbons,
    # bromocarbons, hydrobromocarbons, and halons
    'CFC-11', 'CFC-112', 'CFC-112a', 'CFC-113', 'CFC-113a', 'CFC-114',
    'CFC-114a', 'CFC-115', 'CFC-12', 'CFC-13', 'CH2Cl2',
    'CH3Br', 'CH3CCl3', 'CH3Cl', 'CHCl3',
    'HCFC-124', 'HCFC-133a', 'HCFC-141b', 'HCFC-142b', 'HCFC-22', 'HCFC-31',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
]
len(species_halogen)

49

In [9]:
# uncertainties from IPCC
uncertainty_seed = 38572

unc_ranges = pd.Series({
    k: v / NINETY_TO_ONESIGMA
    for k, v in {
        'CO2': 0.12,
        'CH4': 0.20,
        'N2O': 0.14,
        'O3': 0.50, # total ozone
        'H2O_stratospheric': 1.00, # stratospheric WV from CH4
        'contrails': 0.70, # contrails approx - half-normal
        'BC_on_snow': 1.25, # bc on snow - half-normal
        # 'land_use': 0.50, # land use change
        'land_use': 2./3., # land use change
        'volcanic': 0.25,
        'solar': 0.5, # solar (amplitude)
        **{
            k1: 0.26 * HALOGEN_SCALING if k1 in [
                'HFC-152a', 'CH2Cl2', 'CH3Cl', 'CHCl3',
                'HCFC-133a', 'HCFC-31',
            ] else 0.19 * HALOGEN_SCALING
            for k1 in species_halogen
        },
    }.items()
})

In [10]:
unc_ranges = unc_ranges.reindex(
    ['CO2', 'CH4', 'N2O']
    + species_halogen
    + [
        'O3', 'H2O_stratospheric', 'contrails', 'BC_on_snow', 'land_use',
        'volcanic', 'solar',
    ]
)

In [11]:
n = len(unc_ranges)
df_scale = pd.DataFrame(
    stats.norm.rvs(
        size=(SAMPLES, n), 
        loc=np.ones((SAMPLES, n)), 
        scale=unc_ranges.values, 
        random_state=uncertainty_seed,
    ),
    columns=unc_ranges.index,
)

In [12]:
def f_opt(x, q05_desired, q50_desired, q95_desired):
    "x is (a, loc, scale) in that order."
    q05, q50, q95 = stats.skewnorm.ppf(
        (0.05, 0.50, 0.95), x[0], loc=x[1], scale=x[2]
    )
    return (q05 - q05_desired, q50 - q50_desired, q95 - q95_desired)

lapsi_params = optimize.root(f_opt, [1, 1, 1], args=(0, 1, 2.25)).x
contrails_params = optimize.root(f_opt, [1, 1, 1], args=(19/57, 1, 98/57)).x
irrigation_params = optimize.root(f_opt, [1.5, 0.5, 1], args=(-1, 1, 2)).x
# -0.1 to +0.05, best -0.05, so -1 to +2 x best

In [13]:
# skewnormal for asymmetric distributions
df_scale['BC_on_snow'] = stats.skewnorm.rvs(
    lapsi_params[0],
    loc=lapsi_params[1],
    scale=lapsi_params[2],
    size=SAMPLES,
    random_state=3701584,
)
df_scale['contrails'] = stats.skewnorm.rvs(
    contrails_params[0],
    loc=contrails_params[1],
    scale=contrails_params[2],
    size=SAMPLES,
    random_state=3701585,
)
df_scale['irrigation'] = stats.skewnorm.rvs(
    irrigation_params[0],
    loc=irrigation_params[1],
    scale=irrigation_params[2],
    size=SAMPLES,
    random_state=13710426,
)
trend_solar = pd.Series(stats.norm.rvs(
    size=SAMPLES, 
    loc=0., 
    scale=0.07/NINETY_TO_ONESIGMA, 
    random_state=uncertainty_seed,
))

In [14]:
df_scale.shape

(100000, 60)

In [15]:
g = ds.file.create_group('source/forcing-timeseries/unc/scale')
g.attrs['description'] = 'Scale factors by forcing agent'
for k, v in df_scale.items():
    g.create_dataset(k, data=v.values, chunks=True)

In [16]:
d = ds.file.create_dataset(
    'source/forcing-timeseries/unc/trend_solar',
    data=trend_solar.values, chunks=True,
)
d.attrs['description'] = 'Uncertainty factor of solar trend'

## Input data

### Emissions from Global Carbon Project

In [18]:
path = path_indicator.joinpath('forcing-timeseries/data/gcp_emissions/gcp_2024.csv')
df = pd.read_csv(path, index_col=0)

In [19]:
desc = 'Emissions of CO2 from {} to {}'.format(
    df.index[0], df.index[-1],
)
desc

'Emissions of CO2 from 1750 to 2024'

In [23]:
g = ds.file.create_group('source/forcing-timeseries/gcp_emissions')
g.attrs['description'] = desc
d = g.create_dataset('time', data=df.index.values)
d.attrs['units'] = 'yr'
for k, v in df.items():
    d = g.create_dataset(k, data=v.values)
    d.attrs['units'] = 'Gt C yr-1'

### GHG concentrations

In [26]:
path = path_indicator.joinpath('forcing-timeseries/output/ghg_concentrations_1750-2024.csv')
df = pd.read_csv(path, index_col=0)

In [27]:
# interpolation between 1750 and 1850 will be conducted later
df

,CO2,CH4,N2O,HFC-134a,HFC-23,HFC-32,HFC-125,HFC-143a,HFC-152a,HFC-227ea,...,i-C6F14,C7F16,C8F18,CFC-112,CFC-112a,CFC-113a,CFC-114a,HCFC-133a,HCFC-31,HCFC-124
YYYY,,,,,,,,,,,,,,,,,,,,,
1750,278.377857,729.200000,270.100000,0.00,0.00,0.000218,0.00,0.00,0.00,0.000004,...,0.0000,0.00000,0.000,0.0,0.0000,0.0,0.0000,0.00,0.0000,0.000
1850,285.583545,807.600000,272.100000,0.00,0.00,0.000218,0.00,0.00,0.00,0.000004,...,0.0000,0.00000,0.000,0.0,0.0000,0.0,0.0000,0.00,0.0000,0.000
1851,285.705553,807.768974,272.181132,0.00,0.00,0.000221,0.00,0.00,0.00,0.000004,...,0.0000,0.00000,0.000,0.0,0.0000,0.0,0.0000,0.00,0.0000,0.000
1852,285.843588,808.522848,272.262214,0.00,0.00,0.000223,0.00,0.00,0.00,0.000004,...,0.0000,0.00000,0.000,0.0,0.0000,0.0,0.0000,0.00,0.0000,0.000
1853,285.967605,809.768605,272.365269,0.00,0.00,0.000226,0.00,0.00,0.00,0.000004,...,0.0000,0.00000,0.000,0.0,0.0000,0.0,0.0000,0.00,0.0000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,412.440000,1878.700000,333.300000,113.00,33.70,21.500000,32.80,25.50,7.00,1.700000,...,0.0690,0.11984,0.097,0.4,0.0704,1.0,1.0262,0.43,0.0884,0.938
2021,414.700000,1895.600000,334.600000,118.80,34.90,24.900000,36.30,27.10,7.20,1.900000,...,0.0704,0.12132,0.098,0.4,0.0712,1.1,1.0235,0.43,0.0884,0.922
2022,417.070000,1911.800000,335.900000,124.40,35.90,27.700000,40.10,29.20,7.40,2.100000,...,0.0718,0.12280,0.099,0.4,0.0720,1.2,1.0208,0.43,0.0884,0.906


In [28]:
desc = 'Concentrations of GHGs from {} to {}'.format(
    df.index[0], df.index[-1],
)
desc

'Concentrations of GHGs from 1750 to 2024'

In [29]:
map_units = {'CO2': 'ppm', 'CH4': 'ppb', 'N2O': 'ppb'}
g = ds.file.create_group('source/forcing-timeseries/ghg_concentrations')
g.attrs['description'] = desc
d = g.create_dataset('time', data=df.index.values)
d.attrs['units'] = 'yr'
for k, v in df.items():
    d = g.create_dataset(k, data=v.values)
    d.attrs['units'] = map_units.get(k, 'ppt')

### SLCF emissions

In [32]:
path = path_indicator.joinpath('forcing-timeseries/output/slcf_emissions_1750-2024.csv')
df = pd.read_csv(path, index_col=0)

In [33]:
df

,BC,OC,SO2,NOx,CO,NMVOC,NH3,CH4,N2O
1750,2.117613,15.999935,2.347382,19.412419,348.399748,60.847744,6.746764,NaN,NaN
1751,2.093255,15.742623,2.313252,19.193014,343.798221,59.916853,6.678384,NaN,NaN
1752,2.085127,15.590012,2.300284,19.195734,341.680338,59.463534,6.636892,NaN,NaN
1753,2.093251,15.739703,2.312427,19.111957,343.546226,59.860695,6.698018,NaN,NaN
1754,2.121135,16.117545,2.362232,19.260410,349.685845,61.107100,6.830938,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020,6.949436,29.221131,68.227768,122.308705,727.018794,185.329595,66.219291,345.134701,10.892619
2021,7.173922,33.965894,70.870916,125.019717,785.859058,198.995725,68.244617,353.880847,11.234859
2022,6.697069,25.585478,70.639204,123.258034,665.439079,176.386346,66.818689,354.893434,10.876164
2023,7.553665,41.050145,72.683433,128.409919,895.985575,224.071856,71.556562,369.060112,11.609022


In [34]:
desc = 'Emissions of SLCF species from {} to {}'.format(
    df.index[0], df.index[-1],
)
desc

'Emissions of SLCF species from 1750 to 2024'

In [36]:
g = ds.file.create_group('source/forcing-timeseries/slcf_emissions')
g.attrs['description'] = desc
d = g.create_dataset('time', data=df.index.values)
d.attrs['units'] = 'yr'
for k, v in df.items():
    d = g.create_dataset(k, data=v.values)
    # NOx is scaled to NO2
    units = 'Mt {} yr-1'.format(k.replace('NOx', 'NO2'))
    d.attrs['units'] = units

### Solar irradiance

In [38]:
path = path_indicator.joinpath('forcing-timeseries/data/ar6/solar_erf.csv')
d1 = (
    pd.read_csv(path, index_col=0)
    .squeeze()
    .rename_axis(None)
    .rename(None)
)

In [39]:
desc = 'Effective radiative forcing of solar irradiance from {} to {}'.format(
    d1.index[0], d1.index[-1],
)
desc

'Effective radiative forcing of solar irradiance from -6755 to 2299'

In [40]:
g = ds.file.create_group('source/forcing-timeseries/erf_solar')
g.attrs['description'] = desc
d = g.create_dataset('time', data=d1.index.values)
d.attrs['units'] = 'yr'
d = g.create_dataset('value', data=d1.values)
d.attrs['units'] = 'W m-2'

### Volcanic aerosols

In [42]:
path = path_indicator.joinpath('forcing-timeseries/output/volcanic_sAOD_ERF_annual_-9500-2024.csv')
df = pd.read_csv(path, index_col=0).rename(int)

In [43]:
df

,stratospheric_AOD,ERF_sulfate,ERF_H2O,volcanic_ERF
year,,,,
-9499,0.002910,0.246405,0.000000,0.246405
-9498,0.002910,0.246405,0.000000,0.246405
-9497,0.002910,0.246405,0.000000,0.246405
-9496,0.002910,0.246405,0.000000,0.246405
-9495,0.033068,-0.356748,0.000000,-0.356748
...,...,...,...,...
2020,0.011520,0.074220,0.000000,0.074220
2021,0.008610,0.132409,0.000000,0.132409
2022,0.013721,0.030183,0.141522,0.171705


In [44]:
desc = 'Effective radiative forcing of volcanic activity from {} to {}'.format(
    d1.index[0], d1.index[-1],
)
desc

'Effective radiative forcing of volcanic activity from -6755 to 2299'

In [45]:
g = ds.file.create_group('source/forcing-timeseries/erf_volcanic')
g.attrs['description'] = desc
d = g.create_dataset('time', data=df.index.values)
d.attrs['units'] = 'yr'
d = g.create_dataset('value', data=df['volcanic_ERF'].values)
d.attrs['units'] = 'W m-2'

### Contrails

In [47]:
# contrails
# depends on IEA Word Oil Statistics 2023
# path = repo.retrieve('output/contrails_ERF_1930-2023.csv')
path = path_indicator.joinpath('forcing-timeseries/output/contrails_ERF_1930-2024.csv')
d1 = pd.read_csv(path, index_col=0).squeeze().rename_axis(None).rename(None)

In [48]:
desc = (
    'Effective radiative forcing of contrails and contrail-induced cirrus'
    ' from {} to {}'.format(d1.index[0], d1.index[-1])
)
desc

'Effective radiative forcing of contrails and contrail-induced cirrus from 1930 to 2024'

In [49]:
g = ds.file.create_group('source/forcing-timeseries/erf_contrails')
g.attrs['description'] = desc
d = g.create_dataset('time', data=d1.index.values)
d.attrs['units'] = 'yr'
d = g.create_dataset('value', data=d1.values)
d.attrs['units'] = 'W m-2'

## Aerosols calibration results

In [57]:
path = path_indicator.joinpath('forcing-timeseries/data/ar6/table_mean_thornhill_collins_orignames.csv')
df1 = pd.read_csv(path, index_col=0)
df2 = pd.read_csv(path.with_stem('table_std_thornhill_collins_orignames'), index_col=0)
# map_name = {'HC': 'EESC', 'VOC': 'NMVOC'}
map_name = {'HC': 'ODS', 'VOC': 'NMVOC'}
df = pd.concat([
    df1['Aerosol'].rename('mean'),
    df2['Aerosol_sd'].rename('sd'),
], axis=1).rename(map_name)

In [58]:
df

,mean,sd
Species,,
CO2,0.000000,0.000000
CH4,-0.002653,0.002197
N2O,-0.002090,0.002021
ODS,-0.008080,0.001971
NOx,-0.009166,0.004543
NMVOC,-0.002573,0.005065
SO2,-0.234228,0.223907
OC,-0.072143,0.065461
BC,0.144702,0.158984


In [59]:
desc = 'Reference radiative forcing by SLCF species'
g = ds.file.create_group('source/forcing-timeseries/ari_emitted')
g.attrs['description'] = desc
g.attrs['species'] = df.index.tolist()
for k, v in df.items():
    d = g.create_dataset(k, data=v.values)

In [61]:
path = path_indicator.joinpath('forcing-timeseries/data/fair-calibrate-1.4.1/aerosol_cloud.csv')
df = pd.read_csv(path, index_col=0)

In [62]:
df

,aci_scale,Sulfur,BC,OC
E3SM-2-0,-1.443485,7.152779e-02,3.762593e-41,3.518178e-01
HadGEM3-GC31-LL,-0.940580,2.222328e-02,5.954012e-33,3.674180e-02
GFDL-ESM4,-13197.573317,2.544719e-07,2.699782e-06,6.070785e-07
CNRM-CM6-1,-1.496729,6.005013e-03,4.602495e-02,1.113794e-02
CanESM5,-0.856032,1.992724e-02,3.940789e-01,1.281973e-16
NorESM2-LM,-12953.242881,6.681038e-07,2.927736e-89,1.569533e-06
MRI-ESM2-0,-7.743980,7.756472e-04,4.118827e-03,5.907474e-27
MIROC6,-1.025414,7.298451e-03,1.485128e-01,6.270682e-18
IPSL-CM6A-LR,-1.262191,2.660200e-03,2.010784e-16,1.897347e-03
GISS-E2-1-G,-0.584806,8.194178e-03,1.283971e+00,2.548500e-11


In [63]:
desc = 'Forcing uncertainty ensemble related to aerosol-cloud interactions'
g = ds.file.create_group('source/forcing-timeseries/aci_cal')
g.attrs['description'] = desc
g.attrs['models'] = df.index.tolist()
for k, v in df.items():
    d = g.create_dataset(k, data=v.values)

## Observed GMST

In [65]:
path = path_indicator.joinpath('forcing-timeseries/data/gmst/IGCC_GMST_1850-2023.csv')
d1 = (
    pd.read_csv(path, index_col=0)
    .squeeze()
    .rename(int)
    .rename(None)
    .rename_axis(None)
)

In [66]:
d1

1850   -0.060196
1851    0.057304
1852    0.094804
1853    0.052304
1854    0.062304
          ...   
2019    1.234804
2020    1.264804
2021    1.109804
2022    1.149804
2023    1.432304
Length: 174, dtype: float64

In [67]:
desc = (
    'Temperature time series used for temperature feedback of ozone forcing'
    ' from {} to {}'.format(d1.index[0], d1.index[-1])
)
desc

'Temperature time series used for temperature feedback of ozone forcing from 1850 to 2023'

In [68]:
g = ds.file.create_group('source/forcing-timeseries/temp_obs')
g.attrs['description'] = desc
d = g.create_dataset('time', data=d1.index.values)
d.attrs['units'] = 'yr'
d = g.create_dataset('value', data=d1.values)
d.attrs['units'] = 'K'

## Ozone calibration results

In [71]:
df = {}
for k in ['trop', 'strat']:
    path = path_indicator.joinpath(f'forcing-timeseries/data/ozone/skeie_ozone_{k}.csv')
    df[k] = (
        pd.read_csv(path, index_col=0)
        .rename(int, axis=1)
        .rename_axis(None)
    )

In [73]:
map_name = {
    'trop': 'tropospheric',
    'strat': 'stratospheric',
}

for k, v in df.items():
    g = ds.file.create_group(f'source/forcing-timeseries/skeie_ozone_{k}')
    g.attrs['description'] = f'Reference radiative forcing of {map_name[k]} ozone'
    g.attrs['index_model'] = v.index.tolist()
    g.attrs['columns_year'] = v.columns.tolist()
    g.create_dataset('value', data=v.values)

## Land use

In [76]:
path = path_indicator.joinpath('forcing-timeseries/data/land_use/SARF_LCC_Ghimire2014+Ouyang2022+GCB2024.csv')
df_landuse = pd.read_csv(path, index_col=0)
d_irr = pd.read_csv(path.with_stem('Irrigation_ERF_Wells_FAO'), index_col=0).squeeze()

In [78]:
g = ds.file.create_group('source/forcing-timeseries/sarf_landuse')
g.attrs['description'] = 'Land use forcing due to albedo change'
d = g.create_dataset('time', data=df_landuse.index.values)
d.attrs['units'] = 'yr'
for k, v in df_landuse.items():
    d = g.create_dataset(k, data=v.values)
    d.attrs['units'] = 'W m-2'

g = ds.file.create_group('source/forcing-timeseries/erf_irrigation')
g.attrs['description'] = 'Land use forcing due to irrigation'
d = g.create_dataset('time', data=d_irr.index.values)
d.attrs['units'] = 'yr'
d = g.create_dataset('value', data=d_irr.values)
d.attrs['units'] = 'W m-2'

## Data saved

In [82]:
def func(name, obj):
    desc = obj.attrs.get('description')
    if desc is not None:
        print('{}: {}'.format(name, desc))

ds.file['source/forcing-timeseries'].visititems(func)

aci_cal: Forcing uncertainty ensemble related to aerosol-cloud interactions
ari_emitted: Reference radiative forcing by SLCF species
erf_contrails: Effective radiative forcing of contrails and contrail-induced cirrus from 1930 to 2024
erf_irrigation: Land use forcing due to irrigation
erf_solar: Effective radiative forcing of solar irradiance from -6755 to 2299
erf_volcanic: Effective radiative forcing of volcanic activity from -6755 to 2299
gcp_emissions: Emissions of CO2 from 1750 to 2024
ghg_concentrations: Concentrations of GHGs from 1750 to 2024
sarf_landuse: Land use forcing due to albedo change
skeie_ozone_strat: Reference radiative forcing of stratospheric ozone
skeie_ozone_trop: Reference radiative forcing of tropospheric ozone
slcf_emissions: Emissions of SLCF species from 1750 to 2024
temp_obs: Temperature time series used for temperature feedback of ozone forcing from 1850 to 2023
unc/scale: Scale factors by forcing agent
unc/trend_solar: Uncertainty factor of solar trend


In [83]:
ds.close()

[2025-04-18 10:56:54 mce.core] INFO:file datain/ds_historical.h5 closed
